# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from requests.utils import requote_uri

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Kholtoson,50.2981,103.3097,2.53,87,0,4.25,RU,1612710058
1,Thunder Bay,48.4001,-89.3168,-15.00,58,75,13.80,CA,1612710058
2,Najrān,17.4924,44.1277,73.40,31,35,4.61,SA,1612709590
3,Illapel,-31.6308,-71.1653,75.76,38,0,6.42,CL,1612710058
4,Rikitea,-23.1203,-134.9692,77.05,73,92,10.58,PF,1612710059


In [3]:
cities_df.shape

(561, 9)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
#Lets handle Null values
cities_df = cities_df.dropna()

#Fetching location
locations = cities_df[["Lat","Lng"]].astype(float)

#Fetch humidity for weight
Humidity = cities_df["Humidity"].astype(float)


In [6]:
# Create a Humidity Heatmap layer
#Set layout for figure
layout = {
    'width': '960px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'
         }
#set Max intensity based on max humidity
max_intensity = max(Humidity)

#Initialize figure layout
fig = gmaps.figure(layout=layout,zoom_level=2.15,center=(19,23))

#Set heatmap
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=Humidity,
                                 dissipating=False,
                                 max_intensity=max_intensity,
                                 point_radius = 2)
 #Add heatmap layer to figure  
fig.add_layer(heat_layer)
                                 
#Display Figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='3px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
Ideal_df = cities_df[(cities_df["Max Temp(F)"]<80)
                     &(cities_df["Max Temp(F)"]>70) 
                     & (cities_df["Wind Speed"]<10)
                     & (cities_df["Cloudiness"] == 0)]
Ideal_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Illapel,-31.6308,-71.1653,75.76,38,0,6.42,CL,1612710058
1,Marzuq,14.4000,46.4667,70.43,39,0,3.89,YE,1612709506
2,Castro,-24.7911,-50.0119,72.48,47,0,3.56,BR,1612709672
3,Ancud,-41.8697,-73.8203,71.60,49,0,3.44,CL,1612709985
4,Noumea,-22.2763,166.4572,78.01,100,0,2.30,NC,1612710116
5,Dwārka,22.2394,68.9678,71.83,52,0,8.43,IN,1612710128
6,Assiut,27.1810,31.1837,71.60,46,0,9.22,EG,1612709607


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = Ideal_df.reset_index(drop = True)

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] =""
hotel_df.head(3)

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Illapel,-31.6308,-71.1653,75.76,38,0,6.42,CL,1612710058,
1,Marzuq,14.4000,46.4667,70.43,39,0,3.89,YE,1612709506,
2,Castro,-24.7911,-50.0119,72.48,47,0,3.56,BR,1612709672,


In [9]:
#Set parameters to search for hotels with 5000 meters.
# geocoordinates
# set up a parameters dictionary
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Set the base URl
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [10]:
hotel_df

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Illapel,-31.6308,-71.1653,75.76,38,0,6.42,CL,1612710058,Hotel el Estero
1,Marzuq,14.4000,46.4667,70.43,39,0,3.89,YE,1612709506,Shabwah Hotel
2,Castro,-24.7911,-50.0119,72.48,47,0,3.56,BR,1612709672,ibis Ponta Grossa
3,Ancud,-41.8697,-73.8203,71.60,49,0,3.44,CL,1612709985,Panamericana Hotel Ancud
4,Noumea,-22.2763,166.4572,78.01,100,0,2.30,NC,1612710116,Escapade Îlot Maître
5,Dwārka,22.2394,68.9678,71.83,52,0,8.43,IN,1612710128,Lemon Tree Premier
6,Assiut,27.1810,31.1837,71.60,46,0,9.22,EG,1612709607,نادي و فندق القوات المسلحة


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
# Create a map using City coordinates to set markers
# Create a marker_layer using the name , city and country to fill the info box
fig = gmaps.figure(layout=layout,zoom_level=2.4,center=(15,20))
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

#add heat layer
fig.add_layer(heat_layer)

#Add merkers
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='3px', wi…